In [13]:
import pandas as pd

K = 25

In [14]:
# This function grabs data from Excel worksheet and transforms it into matches.
# Each match has a list of players and corresponding scores.
# There may be from 2 to 5 players in a match (inclusive).

def parser(file_name, sheet_name):
    df = pd.read_excel(file_name, sheet_name=sheet_name, header=None)
    matches = []
    player_names = []
    scores = []

    for i, row in df.iterrows():
        player = row[0]  # Column A (Player)
        score = row[1]   # Column B (Score)

        if pd.isna(player) and pd.isna(score):
            if len(player_names) in [2, 3, 4, 5]:
                matches.append({'player_names': player_names, 'scores': scores})
            
            player_names = []
            scores = []
        else:
            player_names.append(player)
            scores.append(score)

    # Ensure any leftover data is added as the last match
    if len(player_names) in [2, 3, 4, 5]:
        matches.append({'player_names': player_names, 'scores': scores})

    print(f"{sheet_name} uploaded. Total number of matches is: {len(matches)}")
    return matches

In [15]:
# returns expected score for player A with rating Ra
# expeced score is win probability + 0.5 * draw probability
def calculate_expected_score(Ra, Rb):
    Ea = 1 / (1 + pow(10, (Rb-Ra)/400))
    return Ea

In [16]:
# (player_id : name)
players = pd.read_excel("Data.xlsx", sheet_name="Players", header=None)
players.columns = players.iloc[0]
players = players.drop(0).reset_index(drop=True)

# Convert to dictionary (ID as key, Player as value)
id_to_player_dict = pd.Series(players['Player'].values, index=players['ID']).to_dict()

player_to_id_dict = {v: k for k, v in id_to_player_dict.items()}
    
# (player_id : rating)
ratings_dict = {i: 1500 for i in range(1, len(id_to_player_dict)+1)}

matches_dict = {}

In [17]:
# we'll need a dictionary with current ratings, something like a database for 100+ players
# we also need to have a function that calculates updated ratings based on a match

# gets a list of players (in fact player ids and corresponding points in a match and then
# updates elo for all of them

def update_elo(players, scores):
    expected_points = [0] * len(players)
    actual_points = [0] * len(players)
    ratings_before = [0] * len(players)
    ratings_after = [0] * len(players)
    delta = [0] * len(players)
    for idx_A, player_A in enumerate(players):
        for idx_B, player_B in enumerate(players):
            if idx_A != idx_B:
                expected_points[idx_A] += calculate_expected_score(ratings_dict[player_A], ratings_dict[player_B])
                actual_points[idx_A] += 1 * int(scores[idx_A] > scores[idx_B]) + 0.5 * int(scores[idx_A] == scores[idx_B])
    
    for idx_A, player in enumerate(players):
        ratings_before[idx_A] = ratings_dict[player]
        ratings_dict[player] += K * (actual_points[idx_A] - expected_points[idx_A])
        ratings_after[idx_A] = ratings_dict[player]
        delta[idx_A] = ratings_after[idx_A] - ratings_before[idx_A]
        
    dataframe_data = {
        "Players": [id_to_player_dict[player] for player in players],
        "Scores": scores,
        "EP": expected_points,
        "AP": actual_points,
        "RB": ratings_before,
        "RA": ratings_after,
        "Delta": delta
    }
    
    match_df = pd.DataFrame(dataframe_data)
    

    match_df["EP"] = match_df["EP"].round(2)
    match_df["AP"] = match_df["AP"].round(2)
    match_df["RB"] = match_df["RB"].round(2)
    match_df["RA"] = match_df["RA"].round(2)
    match_df["Delta"] = match_df["Delta"].round(2)
    
    return match_df



In [18]:
# Calculating elo

tournament_sequence = [
    "Vuelta_1",
    "Bez_Povoda",
    "Poligon",
    "USA",
    "Nevermore_1",
    "Nevermore_2",
    "Vuelta_2",
    "BG_1"
]

for tournament in tournament_sequence:
    matches = parser("Data.xlsx", tournament)
    for match in matches:
        players = [player_to_id_dict[player] for player in match['player_names']]
        scores = match['scores']
        print(update_elo(players, scores))

Vuelta_1 uploaded. Total number of matches is: 48
             Players  Scores   EP   AP    RB      RA  Delta
0  Владимир Грамагин   130.0  1.5  2.0  1500  1512.5   12.5
1     Юрий Корчемкин   180.0  1.5  3.0  1500  1537.5   37.5
2    Шакир Мамедзаде    40.0  1.5  1.0  1500  1487.5  -12.5
3      Петр Мартынов    20.0  1.5  0.0  1500  1462.5  -37.5
            Players  Scores   EP   AP    RB      RA  Delta
0    Бахтияр Нейман   110.0  1.5  1.0  1500  1487.5  -12.5
1    Сергей Лобачёв   180.0  1.5  3.0  1500  1537.5   37.5
2  Илья Устиловский   -30.0  1.5  0.0  1500  1462.5  -37.5
3    Леонид Койфман   150.0  1.5  2.0  1500  1512.5   12.5
             Players  Scores   EP   AP    RB      RA  Delta
0     Мария Конакова   -10.0  1.5  0.5  1500  1475.0  -25.0
1  Григорий Гольберг   100.0  1.5  2.0  1500  1512.5   12.5
2    Ефим Подвойский   130.0  1.5  3.0  1500  1537.5   37.5
3     Антон Грибанов   -10.0  1.5  0.5  1500  1475.0  -25.0
          Players  Scores   EP   AP    RB      RA  Delt

In [19]:
# showing sorted rating

sorted_rating = [[id_to_player_dict[id], ratings_dict[id]] for id in id_to_player_dict]
sorted_rating = sorted(sorted_rating, key = lambda x: -x[1])
for player, elo in sorted_rating:
    print(f"{player}: {elo:.0f}")

Юрий Корчемкин: 1797
Вадим Барановский: 1772
Сергей Лобачёв: 1722
Николай Арчак: 1703
Шакир Мамедзаде: 1696
Станислав Алимпов: 1685
Евгений Затуловский: 1658
Андрей Романчев: 1653
Владимир Грамагин: 1629
Жора Яковлев: 1619
Григорий Смыслов: 1608
Илья Миронов: 1602
Иван Суманеев : 1600
Алексей Чернов: 1594
Григорий Львович: 1591
Ирина Соколова: 1581
Макс Галкин: 1579
Евгений Новиков: 1549
Леонид Койфман: 1545
Юлия Жоголева: 1541
Арнур Нигметов: 1540
Константин Бриф: 1528
Слава Бельков: 1528
Роман Козелов: 1521
Сергей Меньшиков: 1516
Виктор Исаев: 1515
Евгения Никушина: 1514
Алена Хрисанова: 1511
Ефим Подвойский: 1508
Тимур Мухаматулин: 1507
Руслан Лепшоков: 1505
Марианна Выдревич: 1504
Григорий Гольберг: 1500
Илья Устиловский: 1499
Владимир Пузырев: 1474
Денис Чебыкин: 1473
Туриан Рафаэлли: 1471
Олжас Усипбаев: 1464
Мария Корчемкина: 1461
Борис Вейцман: 1451
Захар Левентул: 1451
Иван Светлаков: 1446
Антон Грибанов: 1446
Самуил Кеслер: 1445
Александр Иванов: 1442
Полина Митриковская: 144